# First load adapter and base model to merge them togther then save it to convert it to bf16 and quatize to 4 and 5 bit gguf

In [ ]:
!pip install -q transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 32.6 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import torch
import warnings

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [ ]:
warnings.filterwarnings( "ignore" )

baseModel = "meta-llama/Llama-2-7b-chat-hf"
peft      = "Elkhayyat17/qlora-med-llama2"
outdir    = "/content/merged"

In [ ]:
from huggingface_hub import login
login()

In [ ]:
print( f"Loading base model: {baseModel}")
model = AutoModelForCausalLM.from_pretrained(
    baseModel,
    torch_dtype = torch.float16,
    device_map = "cuda"
)

Loading base model: meta-llama/Llama-2-7b-chat-hf


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
print( f"Loading PEFT: {peft}" )
model = PeftModel.from_pretrained( model, peft )
print( "Running merge_and_unload" )
model = model.merge_and_unload()

Loading PEFT: Elkhayyat17/qlora-med-llama2


adapter_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Running merge_and_unload


In [ ]:

tokenizer = AutoTokenizer.from_pretrained( baseModel )

if os.path.isdir( outdir ):
    shutil.rmtree( outdir )

model.save_pretrained( outdir )
tokenizer.save_pretrained( outdir )
print( f"Model saved to {outdir}" )

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Model saved to /content/merged


In [ ]:
#create repo for merged model
model.push_to_hub(repo_id="merged-sakina")

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Elkhayyat17/merged-sakina/commit/e69c449dc4337a1012425b1eea30f4c0aa06b2d2', commit_message='Upload LlamaForCausalLM', commit_description='', oid='e69c449dc4337a1012425b1eea30f4c0aa06b2d2', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# install necessary libirary to quatization

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!python3 -m pip install  -r /llama.cpp/requirements.txt


In [ ]:
!python3 llama.cpp/convert.py  /content/merged


In [ ]:
!./llama.cpp/quantize /content/merged/ggml-model-f16.gguf sakina-Q4_K_M.gguf Q4_K_M


ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
main: build = 2232 (7fe4678b)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/merged/ggml-model-f16.gguf' to 'sakina-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /content/merged/ggml-model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32   

# Upload quatizatoin file to HF

In [ ]:

from huggingface_hub import login
login()

In [ ]:
#Create repo first manullay or coding
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="/content/sakina-Q4_K_M.gguf", ## fil  path
    path_in_repo="sakina-Q4_K_M.gguf", ## file name
    repo_id="Elkhayyat17/Sakina-GGUF",
    repo_type="model",
)

sakina-Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Elkhayyat17/Sakina-GGUF/commit/2d4e3fea247fc3ab14bf19e06c14ebbd5a52d542', commit_message='Upload sakina-Q4_K_M.gguf with huggingface_hub', commit_description='', oid='2d4e3fea247fc3ab14bf19e06c14ebbd5a52d542', pr_url=None, pr_revision=None, pr_num=None)